In [1]:
import pandas as pd 

# leitura dos dados em 'data/rocksamples.csv'
rock_samples = pd.read_csv('data/rocksamples.csv')

rock_samples.head() # impressão das 5 primeiras linhas do DataFrame

rock_samples.info() # leitura das informações do sumário
print()
print()

# conversão da massa das amostrasm pra quilo, para facilitar as análises.
rock_samples['Weight (g)'] = rock_samples['Weight (g)'].apply(lambda x : x * 0.001)

# renomeia a coluna, mudando de gramas para quilos
rock_samples.rename(columns={'Weight (g)':'Weight (kg)'}, inplace=True)

rock_samples.head() # impressão das 5 primeiras linhas do dataframe

# cria um novo DataFrame chamado 'missions' como sumário dos dados de cada missão.
missions = pd.DataFrame()

# cada linha da coluna mission criada mostrará dados de uma missão dierente, coletados do DataFrame "rock_samples"
missions['Mission'] = rock_samples['Mission'].unique()

missions.head() # impressão das 5 primeiras linhas do DataFrame

missions.info()# leitura das informações do sumário
print()
print()

# "adiciona" nova coluna ao DataFrame 'missions' onde aparecerá a soma de todas as amostras coletadas em cada missão.
sample_total_weight = rock_samples.groupby('Mission')['Weight (kg)'].sum() # agrupa
missions = pd.merge(missions, sample_total_weight, on='Mission') # mescla dados da coluna "Weight" relativos a cada missão.
missions.rename(columns={'Weight (kg)':'Sample weight (kg)'}, inplace=True) # renomeia coluna
missions # printa o DataFrame inteiro (apenas 6 linhas)

# adiciona (cria) uma coluna que apresenta a diferenção de peso entre as amostras coletadas em viagens subsequentes.
missions['Weight diff'] = missions['Sample weight (kg)'].diff()
missions

# retorna "0" no lugar de Nan, para valores zerados
missions['Weight diff'] = missions['Weight diff'].fillna(value=0)
missions

# cria colunas com nome, mass e diferença de massa para cada modulo da espaçonave.
# modulo lunar
missions['Lunar module (LM)'] = ['Eagle (LM-5)', 'Intrepid (LM-6)', 'Antares (LM-8)', 'Falcon (LM-10)', 'Orion (LM-11)', 'Challenger (LM-12)']
missions['LM mass (kg)'] = [15103, 15235, 15264, 16430, 16445, 16456]
missions['LM mass diff'] = missions['LM mass (kg)'].diff()
missions['LM mass diff'] = missions['LM mass diff'].fillna(value=0)

# modulo de comando
missions['Command module (CM)'] = ['Columbia (CSM-107)', 'Yankee Clipper (CM-108)', 'Kitty Hawk (CM-110)', 'Endeavor (CM-112)', 'Casper (CM-113)', 'America (CM-114)']
missions['CM mass (kg)'] = [5560, 5609, 5758, 5875, 5840, 5960]
missions['CM mass diff'] = missions['CM mass (kg)'].diff()
missions['CM mass diff'] = missions['CM mass diff'].fillna(value=0)
missions

# apresenta peso total da espaçonave em cada missão
missions['Total weight (kg)'] = missions['LM mass (kg)'] + missions['CM mass (kg)']
missions['Total weight diff'] = missions['LM mass diff'] + missions['CM mass diff']
missions

# relação amostra/peso (criando colunas e retornando valores relacionados ao valor informado em "saturnVPayload")
saturnVPayload = 43500
missions['Crewed area : Payload'] = missions['Total weight (kg)'] / saturnVPayload
missions['Sample : Crewed area'] = missions['Sample weight (kg)'] / missions['Total weight (kg)']
missions['Sample : Payload'] = missions['Sample weight (kg)'] / saturnVPayload
missions

# salvando as relações (médias) - function mean()
crewedArea_payload_ratio = missions['Crewed area : Payload'].mean()
sample_crewedArea_ratio = missions['Sample : Crewed area'].mean()
sample_payload_ratio = missions['Sample : Payload'].mean()
print(crewedArea_payload_ratio)
print(sample_crewedArea_ratio)
print(sample_payload_ratio)
print()
print()

# criando DataFrame para a missão Artemis
artemis_crewedArea = 26520
artemis_mission = pd.DataFrame({'Mission':['artemis1','artemis1b','artemis2'],
                                 'Total weight (kg)':[artemis_crewedArea,artemis_crewedArea,artemis_crewedArea],
                                 'Payload (kg)':[26988, 37965, 42955]})
artemis_mission

# estimativa das massa de amostras, baseada nas relações de peso feitas com dados das missões anteriores
artemis_mission['Sample weight from total (kg)'] = artemis_mission['Total weight (kg)'] * sample_crewedArea_ratio
artemis_mission['Sample weight from payload (kg)'] = artemis_mission['Payload (kg)'] * sample_payload_ratio
artemis_mission

# média das estimativas
artemis_mission['Estimated sample weight (kg)'] = (artemis_mission['Sample weight from payload (kg)'] + artemis_mission['Sample weight from total (kg)'])/2
artemis_mission

# determinar quantidade de amostra restante
rock_samples['Remaining (kg)'] = rock_samples['Weight (kg)'] * (rock_samples['Pristine (%)'] * .01)
rock_samples.head()

rock_samples.describe() # resumo/média ds valores no DataFrame "rock_samples"

# verificando quais amostras estão com baixo estoque
low_samples = rock_samples.loc[(rock_samples['Weight (kg)'] >= .16) & (rock_samples['Pristine (%)'] <= 50)]
low_samples.head()

low_samples.info()
print()
print()

# verificar quais dos tipos únicos de amostras estão com estoque baixo
low_samples.Type.unique()

# verificar quais os tipos únicos de amostras existentes
rock_samples.Type.unique()

# verificar quantidade de amostras de cada tipo único com baixo estoque
low_samples.groupby('Type')['Weight (kg)'].count()

# verificando númro de amostras com baixo estoque, nos tipos especificados
needed_samples = low_samples[low_samples['Type'].isin(['Basalt', 'Breccia'])]
needed_samples.info()
print()
print()

# comparando a massa total das amostras necessárias com os valores em "rock_samples"
needed_samples.groupby('Type')['Weight (kg)'].sum() # valores atuais, dos tipos com baixo estoque

rock_samples.groupby('Type')['Weight (kg)'].sum() # valores de todos os tipos, inicialmente.

# adicionando o tipo "crustal" às necessárias, já que originalmente, a quantidade de amostra já era baixa
needed_samples = pd.concat([needed_samples,rock_samples.loc[rock_samples['Type'] == 'Crustal']])
needed_samples.info()
print()
print()

# criando um novo DataFrame para amostras necesárias
needed_samples_overview = pd.DataFrame()
needed_samples_overview['Type'] = needed_samples.Type.unique()
needed_samples_overview

# valores coletados de cada tipo, originalmente
needed_sample_weights = needed_samples.groupby('Type')['Weight (kg)'].sum().reset_index()
needed_samples_overview = pd.merge(needed_samples_overview, needed_sample_weights, on='Type')
needed_samples_overview.rename(columns={'Weight (kg)':'Total weight (kg)'}, inplace=True)
needed_samples_overview

# retornando o peso médio de cada amostra de cada tipo
needed_sample_ave_weights = needed_samples.groupby('Type')['Weight (kg)'].mean().reset_index()
needed_samples_overview = pd.merge(needed_samples_overview, needed_sample_ave_weights, on='Type')
needed_samples_overview.rename(columns={'Weight (kg)':'Average weight (kg)'}, inplace=True)
needed_samples_overview

# add o número de amostras coletadas e porcentagens
total_rock_count = rock_samples.groupby('Type')['ID'].count().reset_index()
needed_samples_overview = pd.merge(needed_samples_overview, total_rock_count, on='Type')
needed_samples_overview.rename(columns={'ID':'Number of samples'}, inplace=True)
total_rocks = needed_samples_overview['Number of samples'].sum()
needed_samples_overview['Percentage of rocks'] = needed_samples_overview['Number of samples'] / total_rocks
needed_samples_overview

# determinando a média total estimada de peso das amostras
artemis_ave_weight = artemis_mission['Estimated sample weight (kg)'].mean()
artemis_ave_weight

# We can use this number to determine how many of each rock we want the astronauts to aim to collect:
needed_samples_overview['Weight to collect'] = needed_samples_overview['Percentage of rocks'] * artemis_ave_weight
needed_samples_overview['Rocks to collect'] = needed_samples_overview['Weight to collect'] / needed_samples_overview['Average weight (kg)']
needed_samples_overview


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2229 entries, 0 to 2228
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID            2229 non-null   int64  
 1   Mission       2229 non-null   object 
 2   Type          2229 non-null   object 
 3   Subtype       2226 non-null   object 
 4   Weight (g)    2229 non-null   float64
 5   Pristine (%)  2229 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 104.6+ KB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Mission  6 non-null      object
dtypes: object(1)
memory usage: 180.0+ bytes


0.4963026819923371
0.002848764392685611
0.0014369195019157087


<class 'pandas.core.frame.DataFrame'>
Index: 27 entries, 11 to 2183
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------      

,Type,Total weight (kg),Average weight (kg),Number of samples,Percentage of rocks,Weight to collect,Rocks to collect
0,Basalt,17.42340,1.244529,351,0.258850,16.467267,13.231731
1,Breccia,10.11850,1.264813,959,0.707227,44.991764,35.571884
2,Crustal,4.74469,0.103145,46,0.033923,2.158103,20.922917
